<a href="https://colab.research.google.com/github/siddhartha1506/pandas-quiz/blob/main/Mall_customer_segmentation_using_KMeans_clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler 
from sklearn.cluster import KMeans

In [ ]:
path = '/content/sample_data/Mall_Customers.csv'

In [ ]:
df = pd.read_csv(path)

In [ ]:
df.head()

,CustomerID,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40


In [ ]:
df.dtypes

CustomerID                 int64
Gender                    object
Age                        int64
Annual Income (k$)         int64
Spending Score (1-100)     int64
dtype: object

In [ ]:
#customerID not important
df = df.drop('CustomerID',axis=1)

In [ ]:
df.describe()

,Age,Annual Income (k$),Spending Score (1-100)
count,200.000000,200.000000,200.000000
mean,38.850000,60.560000,50.200000
std,13.969007,26.264721,25.823522
min,18.000000,15.000000,1.000000
25%,28.750000,41.500000,34.750000
50%,36.000000,61.500000,50.000000
75%,49.000000,78.000000,73.000000
max,70.000000,137.000000,99.000000


To check null values

In [ ]:
df.isnull().sum()

Gender                    0
Age                       0
Annual Income (k$)        0
Spending Score (1-100)    0
dtype: int64

**Exploratory Data Analysis**

Gender

In [ ]:
df['Gender'].value_counts()

Female    112
Male       88
Name: Gender, dtype: int64

In [ ]:
fig1 = px.pie(df,names='Gender',title='Ratio of Females vs Males')
fig1.show()

Age

In [ ]:
fig2 = px.box(df,x='Gender',y='Age',color='Gender')
fig2.show()

Minimum age for both males and females is 18.
The average age for males is 37 and for females is 35.

In [ ]:
fig3 = px.histogram(df,x='Age',nbins=15,color='Gender',title='Age group with most number of customers')
fig3.show()

Age group 30-34 has more customers,followed by 35-39 and 45-49.

In [ ]:
fig4 = px.histogram(df,x='Age',y='Spending Score (1-100)',color='Gender',nbins=15,title='Spending Score for Age group')
fig4.show()

Age group 30-34 has a higher spending Score,followed by 35-39 and 20-24

In [ ]:
fig5 = px.histogram(df,x='Age',y='Annual Income (k$)',nbins=15,color='Gender',title='Annual Income Age Group Wise')
fig5.show()

Age group 30-34 has a higher Annual Income,followed by 35-39 and 45-49.

In [ ]:
fig6 = px.scatter(df, x='Annual Income (k$)',y='Spending Score (1-100)',color='Age',title= 'Annual Income vs Spending Score Scatter Plot')
fig6.show()

Young people have spend more but have less income.                
Age group 30-34 & 35-39 have a high Annual Income and also have a High Spending Score and have a higher count in the number of customers.
So Age group 30-34 are regular customers and they should be the utmost priority.

**Customer Segmentation using K-Means**

In [ ]:
df.head()

,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,Male,19,15,39
1,Male,21,15,81
2,Female,20,16,6
3,Female,23,16,77
4,Female,31,17,40


**Data Standardisation**

Assign numerical values to categorical data

In [ ]:
df['Gender'] = df['Gender'].map({'Male':1,'Female':0})

In [ ]:
df.head()

,Gender,Age,Annual Income (k$),Spending Score (1-100)
0,1,19,15,39
1,1,21,15,81
2,0,20,16,6
3,0,23,16,77
4,0,31,17,40


In [ ]:
X = df.copy()
X = StandardScaler().fit(X).transform(X)
X[0:5]

array([[ 1.12815215, -1.42456879, -1.73899919, -0.43480148],
       [ 1.12815215, -1.28103541, -1.73899919,  1.19570407],
       [-0.88640526, -1.3528021 , -1.70082976, -1.71591298],
       [-0.88640526, -1.13750203, -1.70082976,  1.04041783],
       [-0.88640526, -0.56336851, -1.66266033, -0.39597992]])

**To find the best value of k**

**Elbow method**

In [ ]:
wcss=[]
for i in range(1,21):
    km=KMeans(n_clusters=i)
    km.fit(X)
    wcss.append(km.inertia_)

In [ ]:
fig = px.line(x=np.arange(1,21,1),y=wcss,markers=True)
fig.update_layout(
    title="Elbow Method",
    xaxis_title="K Value",
    yaxis_title="WCSS",
    legend_title="Legend Title",
)
fig.show()

From the above graph we choose 5 as the k values

**KMeans model**

In [ ]:
k_means = KMeans(init="k-means++", n_clusters=5, n_init=15)
k_means.fit(X)

KMeans(n_clusters=5, n_init=15)

In [ ]:
print("Cluster Centers: ",k_means.cluster_centers_)

Cluster Centers:  [[-0.88640526  0.7517978  -0.51757746 -0.4420241 ]
 [-0.88640526 -0.75047453 -0.00501655  0.6979562 ]
 [ 1.12815215 -0.76072691  0.05496398  0.83369302]
 [ 0.25517727  0.0729628   1.14279271 -1.32381522]
 [ 1.12815215  1.22385356 -0.4498575  -0.44231533]]


In [ ]:
df1 = df.copy()

In [ ]:
df1['cluster']  = k_means.labels_
df1['Gender'] = df1['Gender'].map({1:'Male',0:'Female'})
df1.head()

,Gender,Age,Annual Income (k$),Spending Score (1-100),cluster
0,Male,19,15,39,2
1,Male,21,15,81,2
2,Female,20,16,6,0
3,Female,23,16,77,1
4,Female,31,17,40,0


**Visualization of KMeans Cluster**

In [ ]:
fig7 = px.scatter_3d(df1,x='cluster',y='Age',z='Spending Score (1-100)',color ='Annual Income (k$)',title='3-D Visualisation of KMeans')
fig7.show()

In [ ]:
fig8 = px.scatter(df1,x='Annual Income (k$)',y='Spending Score (1-100)',color='cluster',title='2-D Visualisation of KMeans')
fig8.show()

In [ ]:
fig9 = px.scatter_3d(df1,x='cluster',y='Age',z='Spending Score (1-100)',color ='cluster')
fig9.show()

In [ ]:
df1.groupby(['cluster']).mean()

,Age,Annual Income (k$),Spending Score (1-100)
cluster,,,
0,49.325581,47.000000,38.813953
1,28.392857,60.428571,68.178571
2,28.250000,62.000000,71.675000
3,39.866667,90.500000,16.100000
4,55.903226,48.774194,38.806452


Clusters 0 and 1 have similar Age Group(28) with similar Annual Income and higher Spending Score                            
Cluster 3 with Age Group(40) have highest Annual Income but less Spending Score